In [1]:
#Librairies
import numpy as np
import pandas as pd

from src.preprocessing import apply_transformation

In [8]:
import sys

# Obtient le nom du fichier
nom_fichier = sys.argv[0]
print(nom_fichier)

/opt/conda/lib/python3.12/site-packages/ipykernel_launcher.py


In [4]:
#Data principale
fred_md = pd.read_csv("Macroeconometrics/data/fred_md_2024_12.csv")
#Metadata
fred_info = pd.read_csv("Macroeconometrics/data/FRED_MD_updated_appendix.csv", encoding="latin1")

fred_md.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Macroeconometrics/data/fred_md_2024_12.csv'

In [ ]:
#Mise des dates en index
fred_md_short = (
    fred_md.iloc[1:]
    .assign(sasdate=pd.to_datetime(fred_md.iloc[1:].sasdate, format="%m/%d/%Y"))
    .set_index("sasdate")
)

#Transformation des séries en fonction des metadonées
for _, row in fred_info.iterrows():
    series_name = row['fred']
    transformation_code = row['tcode']

    with contextlib.suppress(Exception):
        fred_md_short[series_name] = apply_transformation(fred_md_short[series_name], transformation_code)